In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-03-22 14:42:34.851319: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-22 14:42:34.961212: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-22 14:42:35.373121: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-22 14:42:36.465685: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Function to load images from a folder
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = tf.keras.preprocessing.image.load_img(
            os.path.join(folder, filename), color_mode="grayscale"
        )
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        images.append(img_array)
    return np.array(images)


# Divide data into training and testing sets
def split_data(folder, test_size=0.2):
    filenames = os.listdir(folder)
    np.random.shuffle(filenames)
    train_files, test_files = train_test_split(filenames, test_size=test_size)
    return train_files, test_files


# Custom image generator for loading images from a list of file paths
def custom_image_generator(file_paths, batch_size=32, target_size=(640, 480)):
    num_samples = len(file_paths)
    while True:
        for offset in range(0, num_samples, batch_size):
            batch_files = file_paths[offset : offset + batch_size]
            batch_images = []
            batch_labels = []
            for file_path in batch_files:
                img = tf.keras.preprocessing.image.load_img(
                    file_path, color_mode="grayscale", target_size=target_size
                )
                img_array = tf.keras.preprocessing.image.img_to_array(img)
                batch_images.append(img_array)
                # Assuming file name contains class information (e.g., 'noise' or 'signal')
                label = 0 if "noise" in file_path else 1
                batch_labels.append(label)
            yield np.array(batch_images), np.array(batch_labels)

In [4]:
# Load images from folders
noise_folder = "/home/arush/GW_Test_1/Data_Generation/Data/Noise"
signal_folder = "/home/arush/GW_Test_1/Data_Generation/Data/Signal"

noise_files_train, noise_files_test = split_data(noise_folder)
signal_files_train, signal_files_test = split_data(signal_folder)

train_files = [
    os.path.join(noise_folder, filename) for filename in noise_files_train
] + [os.path.join(signal_folder, filename) for filename in signal_files_train]

test_files = [os.path.join(noise_folder, filename) for filename in noise_files_test] + [
    os.path.join(signal_folder, filename) for filename in signal_files_test
]

# Create custom image generators
train_generator = custom_image_generator(train_files)
test_generator = custom_image_generator(test_files)

In [6]:
batch_size = 32

# Define CNN model
model = tf.keras.Sequential(
    [
        tf.keras.layers.Conv2D(
            32, (3, 3), activation="relu", input_shape=(640, 480, 1)
        ),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model using generator
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_files) // batch_size,
    epochs=10,
    validation_data=test_generator,
    validation_steps=len(test_files) // batch_size,
)

# Evaluate the model
test_loss, test_acc = model.evaluate(
    test_generator, steps=len(test_files) // batch_size
)
print("Test Accuracy:", test_acc)

/home/arush/anaconda3/envs/python39_1/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
137/137 ━━━━━━━━━━━━━━━━━━━━ 850s 6s/step - accuracy: 0.9587 - loss: 158.0500 - val_accuracy: 0.0809 - val_loss: 53312.3086
Epoch 2/10
137/137 ━━━━━━━━━━━━━━━━━━━━ 853s 6s/step - accuracy: 0.9044 - loss: 3389.6836 - val_accuracy: 0.0637 - val_loss: 49090.0820
Epoch 3/10
137/137 ━━━━━━━━━━━━━━━━━━━━ 851s 6s/step - accuracy: 0.8254 - loss: 3784.4727 - val_accuracy: 0.0637 - val_loss: 3691.4070
Epoch 4/10
137/137 ━━━━━━━━━━━━━━━━━━━━ 860s 6s/step - accuracy: 0.8970 - loss: 308.2911 - val_accuracy: 0.9288 - val_loss: 0.6875
Epoch 5/10
137/137 ━━━━━━━━━━━━━━━━━━━━ 860s 6s/step - accuracy: 0.6663 - loss: 1.2303 - val_accuracy: 0.9064 - val_loss: 0.6265
Epoch 6/10
137/137 ━━━━━━━━━━━━━━━━━━━━ 858s 6s/step - accuracy: 0.8538 - loss: 0.6247 - val_accuracy: 0.9064 - val_loss: 0.5823
Epoch 7/10
137/137 ━━━━━━━━━━━━━━━━━━━━ 859s 6s/step - accuracy: 0.8302 - loss: 0.5965 - val_accuracy: 0.9064 - val_loss: 0.5477
Epoch 8/10
137/137 ━━━━━━━━━━━━━━━━━━━━ 858s 6s/step - accuracy: 0.8078 - lo